In [2]:
import sequencer2
import pandas as pd
import re
import ast
import numpy as np
import pandas as pd
import warnings
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

def convertToVGrade(number):
    ''' Converts 3.0, 4.0, 5.0, etc., to V3, V4, V5, etc. '''
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=FutureWarning)
        # Convert to int
        number = int(number)
    return 'V' + str(number)

def preprocess_input(input_data):
    hold_types = ["Small Crimp", "Medium Crimp", "Large Crimp", "Pocket", "Small Pocket", "Jug", "Small Pinch", "Medium Pinch", "Large Pinch", "Slopy Pinch", "Sloper"]
    
    encoded_seq = []
    
    for seq in input_data:
        current_hold1, current_hold2, target_hold, distance = seq
        encoded_hold = [0] * (len(hold_types) + 1)  # +1 for the distance
        
        if current_hold1 in hold_types:
            encoded_hold[hold_types.index(current_hold1)] = 1
        if current_hold2 in hold_types:
            encoded_hold[hold_types.index(current_hold2)] = 1
        if target_hold in hold_types:
            encoded_hold[hold_types.index(target_hold)] = 1
        
        # Append the distance as the last element of the encoded sequence
        encoded_hold[-1] = distance
        
        encoded_seq.append(encoded_hold)
    
    # Pad sequences with masking to a length of 12
    max_seq_length = 12
    padded_input = pad_sequences([encoded_seq], padding='post', dtype='float32', maxlen=max_seq_length)
    return padded_input

def predict(moves):
    model = load_model("2016Model.h5") 
    padded_input = preprocess_input(moves)
    if padded_input is not None:
        # Predict using the loaded model
        predicted_grade = model.predict(padded_input)
        predicted_grade_index = np.argmax(predicted_grade)
        predicted_grade_label = convertToVGrade(predicted_grade_index + 3)  # Adjust for your grade labels
        return predicted_grade_label
    else:
        return "Invalid input data"

def main():
    # Prompt the user to enter the year
    while True:
        # Prompt the user to enter the year
        year = input("Please enter Moonboard year (2016 or 2017): ")
        if year == '2016' or year == '2017':
            break  # Exit the loop if a valid year is entered
        else:
            print("Please ensure the year entered is either 2016 or 2017")
    
    # Prompt the user to enter start holds
    print("Enter the start holds as a list of hold positions. Separate each position with a space.")
    print("For example: 'F6 H5'")
    start_holds_input = input("Enter start holds: ").upper().split()

    # Create the start_holds list from user input
    start_holds = [{'Position': position} for position in start_holds_input]

    # Prompt the user to enter the rest of the holds
    print("Enter the remaining holds as a list of hold positions. Separate each position with a space.")
    print("For example: 'F8 J11 E13 C16 B18'")
    remaining_holds_input = input("Enter remaining holds: ").upper().split()

    # Create the remaining_holds list from user input
    remaining_holds = [{'Position': position} for position in remaining_holds_input]

    # Combine start holds and remaining holds into holds_data
    holds_data = [start_holds, remaining_holds]

    # Call the function from sequencer2 module
    result = sequencer2.main(holds_data, year)
    
    # Call the predict function to predict the grade
    grade = predict(result)
    print("Predicted Grade:", grade)

if __name__ == "__main__":
    main()

Please enter Moonboard year (2016 or 2017):  2017


Enter the start holds as a list of hold positions. Separate each position with a space.
For example: 'F6 H5'


Enter start holds:  F6 H5


Enter the remaining holds as a list of hold positions. Separate each position with a space.
For example: 'F8 J11 E13 C16 B18'


Enter remaining holds:  F8 J11 E13 C16 B18


1/1 [==============================] - 1s 873ms/step
Predicted Grade: V4
